## 1. Problems encountered in the map

### I have downloaded a map of the New York City area and found some of the problems as follows.

#### First lets find inconsistent postcode.

In [1]:
from pymongo import MongoClient

import re

client = MongoClient("mongodb://localhost:27017")
db = client.osm

postcode_regex = r'^[0-9]{5}$'

Actual number of postcodes

In [2]:
db.nyc.count({"addr.postcode": {"$exists": 1}})

36992

Number of consistent postcodes.

In [3]:
db.nyc.count({"addr.postcode": {"$regex": postcode_regex}})

36964

Lets not look at all the inconsistent postcodes

In [4]:
inconsistent_postcodes = db.nyc.find({"addr.postcode": {"$not": re.compile(postcode_regex), "$exists": 1}})

for item in inconsistent_postcodes:
    print(item['addr']['postcode'])

12
NY 10703
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
NY 10533
08854-8006
08901-2867
08854-8062
08854-8020
08901-8520
08901-8529
08901-8536
08901-8519
08901-2832
08854-8009
08854-8000
08854-8063
08854-8019
08854-8031
08901-8542
08854-8040
08901-8502


There are three types of inconsistencies in the data. The first can be removed by dropping the "NY" and trimming the string for white spaces. And the second can be corrected by dropping the data after the "-". The finally one is invalid postcode.

Lets, correct these two issues.

In [5]:
inconsistent_postcodes = db.nyc.find({"addr.postcode": {"$not": re.compile(postcode_regex), "$exists": 1}})

for item in inconsistent_postcodes:
    postcode = item['addr']['postcode']
    if postcode[:2] == 'NY':
        cleaned_data = postcode[2:].strip()
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.postcode": cleaned_data}})
    elif '-' in postcode:
        cleaned_data = postcode.split('-')[0]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.postcode": cleaned_data}})

#### Now, lets check to see if there are any inconsistencies in the street address

In [6]:
inconsistent_streets = db.nyc.find({"addr.street": {"$exists": 1}}).limit(50)

for item in inconsistent_streets:
    print(item['addr']['street'])

Convent Lane
Monroe St
1st St
Park Ave
Madison St
Jackson St
Clinton St
2nd St
Newark St
4th St
4th St
3rd St
Washington St
Adams St
Garden St
Adams St
Hudson St
7th St
Bloomfield St
Clinton St
Park Ave
Madison St
East 240th Street
Webb Avenue
Bainbridge Avenue
Braddock Avenue
Main Street
White Plains Road
East 2nd Street
Amsterdam Avenue
Front Street
Grand Street
St. Nicholas Avenue
Old Mill Road
Broadway
Ellis Pky
Broadway
North End Avenue
Chambers Street
Speedwell Ave
East 52nd Street
West Beech Street
West 58th Street
56th Street
South Orange Avenue
Bedford Avenue
Washington Avenue
Sterling Place
Nostrand Avenue
Schenectady Avenue


Three abbreviation are clearly noticeable. The first is that "St" is used instead of "Street" and "Ave." is used in place of "Avenue". Finally, "St." need to be changed to "St."

Lets fix these problems

In [7]:
street_regex = "((St|Pky|Ave\.)$)|(^St.\s)"

inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(street_regex)}})

for item in inconsistent_streets:
    print(item['addr']['street'])

Monroe St
1st St
Madison St
Jackson St
Clinton St
2nd St
Newark St
4th St
4th St
3rd St
Washington St
Adams St
Garden St
Adams St
Hudson St
7th St
Bloomfield St
Clinton St
Madison St
St. Nicholas Avenue
Ellis Pky
Springfield Ave.
Franklin Ave.
E 43rd St
4th St
7th St
Bloomfield St
Mitchell St
Mitchell St


Lets fix these two abbreviations.

In [8]:
inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(street_regex)}})

for item in inconsistent_streets:
    street = item['addr']['street']
    if street[-2:] == "St":
        cleaned_data = street + "reet"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[:3] == "St.":
        cleaned_data = "Street" + street[4:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[-4:] == "Ave.":
        cleaned_data = street[:-1] + "nue"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[-3:] == "Pky":
        cleaned_data = street[:-3] + "Parkway"
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})

While querying for the above abbreviations we have found another abbreviation, "E" is used in place of "East". Lets try and look if there are more direction based streets.

In [9]:
direction_street_regex = "^[NSEW]\s"

inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(direction_street_regex)}})

for item in inconsistent_streets:
    print(item['addr']['street'])

W 27th
E 43rd Street


Lets fix this directional the abbreviations.

In [10]:
inconsistent_streets = db.nyc.find({"addr.street": {"$regex": re.compile(direction_street_regex)}})

for item in inconsistent_streets:
    street = item['addr']['street']
    if street[0] == "N":
        cleaned_data = "North" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "S":
        cleaned_data = "South" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "E":
        cleaned_data = "East" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})
    elif street[0] == "W":
        cleaned_data = "West" + street[1:]
        item_id = item['_id']
        db.nyc.update_one({'_id': item_id}, {"$set": {"addr.street": cleaned_data}})

## 2. Data Overview

This section contains basic statistics about the dataset and the MongoDB queries used to gather them.

### File Size
* sample.osm ............ 96.9MB
* sample.osm.json ....... 137.5MB

#### Number of documents

In [11]:
db.nyc.count()

446087

#### Number of nodes

In [12]:
db.nyc.count({"type": "node"})

383534

#### Number of ways

In [13]:
db.nyc.count({"type": "way"})

62525

#### Number of unique users

In [14]:
len(db.nyc.distinct("created.user"))

1663

#### Number of schools

In [21]:
db.nyc.count({"amenity": "school"})

189

#### Number of sport centers

In [15]:
db.nyc.count({"leisure": "sports_centre"})

11

#### Number of restaurants

In [16]:
db.nyc.count({"amenity": "restaurant"})

132

#### Number of French restaurants

In [17]:
db.nyc.count({"amenity": "restaurant", "cuisine": "french"})

6

## 3. Additional Ideas

During the analysis, I had noticed that the data had inconsistencies like abbreviation or incorrect data. This problem can be reduced by using two steps:
1. By specifying the data standards.
2. Having corrective action and validations in the form while inputing the data based on the type of data.


### Additional data exploration using MongoDB queries

                                                
#### Top 10 appearing amenities

In [18]:
items = db.nyc.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity","count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])

for i in items:
    print(i)

{u'count': 300, u'_id': u'parking'}
{u'count': 201, u'_id': u'bicycle_parking'}
{u'count': 192, u'_id': u'place_of_worship'}
{u'count': 189, u'_id': u'school'}
{u'count': 132, u'_id': u'restaurant'}
{u'count': 46, u'_id': u'fast_food'}
{u'count': 42, u'_id': u'cafe'}
{u'count': 34, u'_id': u'bank'}
{u'count': 26, u'_id': u'fire_station'}
{u'count': 26, u'_id': u'toilets'}


#### All the cusine types

In [19]:
db.nyc.distinct("cuisine")

[u'french',
 u'pizza',
 u'american',
 u'falafel',
 u'new_american',
 u'sandwich',
 u'burger',
 u'coffee_shop',
 u'thai',
 u'sushi',
 u'chinese',
 u'chicken',
 u'italian',
 u'regional',
 u'burger;chicken;pizza',
 u'soul_food',
 u'japanese',
 u'seafood',
 u'spanish',
 u'dessert',
 u'donut',
 u'vegan',
 u'mexican',
 u'seafood;steak',
 u'steak',
 u'ice_cream',
 u'vapor_lounge',
 u'caribbean',
 u'bagel',
 u'taiwanese',
 u'international',
 u'American',
 u'indian',
 u'peruvian',
 u'crepe',
 u'korean',
 u'bagel;chicken;american',
 u'greek',
 u'asian',
 u'soup',
 u'irish',
 u'southern',
 u'Columbian',
 u'fish',
 u'Po_Boys',
 u'mediterranean']